In [1]:
import os
import sys
import time
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
# store original policy layer
policy_layer = pd.read_csv("E:\REMM\REMMRun_Max_Sprawl\Input_Files\zoning_parcels_p.csv")

# store table of urban center parcels
urban_centers = pd.DataFrame.spatial.from_featureclass(r"E:\REMM\Boundaries\Center01102019.gdb\Urban_Center_Parcels")
parcel_ids_urb = urban_centers['parcel_id'].tolist()

# store table of agriculture parcels
ag_parcels = pd.DataFrame.spatial.from_featureclass(r"E:\REMM\Boundaries\Center01102019.gdb\Ag_Parcels")
parcel_ids_ag = ag_parcels['parcel_id_REMM'].tolist()

In [3]:
# get parcel ids to filter out
parcels_exclude = parcel_ids_urb + parcel_ids_ag
print(len(parcel_ids_urb))
print(len(parcel_ids_ag))
print(len(parcel_ids_urb + parcel_ids_ag))

# filter out urban center parcels
policy_layer2 = policy_layer[~policy_layer['parcel_id'].isin(parcels_exclude)].copy()

57188
63817
121005


In [4]:
ag_parcels2 = ag_parcels[['parcel_id_REMM', 'max_dua', 'max_far']].copy()
ag_parcels2['year'] = 2025
ag_parcels2['type1'] = 1
ag_parcels2['type2'] = 0
ag_parcels2['type3'] = 0
ag_parcels2['type4'] = 0
ag_parcels2['type5'] = 0
ag_parcels2['type6'] = 0
ag_parcels2['type7'] = 0
ag_parcels2['type8'] = 0
ag_parcels2['locnote'] = 'max_sprawl'
ag_parcels2['mponote'] = 'max_sprawl'

ag_parcels2.rename(columns={'parcel_id_REMM':'parcel_id'}, inplace=True)
ag_parcels2.head(10)

ag_parcels2['max_dua'].fillna(3.5, inplace=True)
ag_parcels2['max_far'].fillna(.4, inplace=True)

In [5]:
# concatenate old policy layer table (minus ag and urban parcels) with new ag parcels, fill no data with 0
policy_layer_ms = pd.concat([policy_layer2, ag_parcels2])
policy_layer_ms.tail()

,parcel_id,max_dua,max_far,year,type1,type2,type3,type4,type5,type6,type7,type8,locnote,mponote
63812,886695,0.025,0.4,2025,1,0,0,0,0,0,0,0,max_sprawl,max_sprawl
63813,886696,0.025,0.4,2025,1,0,0,0,0,0,0,0,max_sprawl,max_sprawl
63814,886697,0.025,0.4,2025,1,0,0,0,0,0,0,0,max_sprawl,max_sprawl
63815,886698,0.025,0.4,2025,1,0,0,0,0,0,0,0,max_sprawl,max_sprawl
63816,886699,0.025,0.4,2025,1,0,0,0,0,0,0,0,max_sprawl,max_sprawl


In [7]:
# Export
policy_layer_ms.to_csv(".\\Outputs\zoning_parcels_p_maxsprawl.csv", index=False)